In [2]:
import syft as sy
import torch as th
import numpy as np

# UNCOMMENT THIS FOR FIRST-TIME SETUP
sy.login().setup(email="owner@myorg.com", password="owerpwd", domain_name="Trask's Node")


# LOGIN AS ROOT USER\
root_domain = sy.login(email="owner@myorg.com", password="owerpwd")

# create a compliance officer user
root_domain.users.create(
        email="compliance@myorg.com", password="pwd123", role="Compliance Officer"
    )

In [3]:
import transformers

# Load model
model = transformers.AutoModel.from_pretrained("distilbert-base-uncased")
tokenizer = transformers.AutoTokenizer.from_pretrained("distilbert-base-uncased")
config = model.config

# Turn off dropout
model = model.eval()

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
model_ptr = model.tag('distilbert').send(root_domain)

In [ ]:
root_domain.store

In [10]:
tokenizer_ptr = tokenizer.tag('tokenizer').send(root_domain)

In [11]:
root_domain.store

,ID,Tags,Description,object_type
0,<UID: 23f6174a7892439c854d77867fa3e59a>,[distilbert],,<class 'transformers.models.distilbert.modelin...


In [13]:
tokenizer_ptr.id_at_location

<UID: 59ff56dbd38841ada4d443c6a2cbbaac>

In [18]:
import datasets
import torch
train_set = datasets.load_dataset('sst', split='train')
train_loader = torch.utils.data.DataLoader(train_set, batch_size=32, shuffle=True)

num_labels = len(set(train_set['label']))
config.num_labels = num_labels

bx = next(iter(train_loader))['sentence']
bx_ptr = sy.lib.python.List(bx).send(root_domain)

In [19]:
# Tokenize on client
bt_ptr = tokenizer_ptr(bx_ptr, padding=True, return_tensors='pt', truncation=True)
print(bt_ptr)

In [21]:
# Forward on client
model_ptr.eval()
out_ptr = model_ptr(bt_ptr['input_ids'], bt_ptr['attention_mask'], return_dict=False)
client_out = out_ptr.get()

Can't deserialize grad with b''
Can't deserialize grad with b''


In [22]:
bt = tokenizer(bx, padding=True, return_tensors='pt', truncation=True)
local_out = model(bt['input_ids'], bt['attention_mask'], return_dict=False)

In [23]:
torch.mean(torch.abs(client_out[0] - local_out[0]))

tensor(0., grad_fn=<MeanBackward0>)